<a href="https://colab.research.google.com/github/kazitoufiq/rest-api-in-R/blob/master/OBD_Data_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Install required packages

In [ ]:
install.packages("httr")
install.packages("dplyr")
install.packages("jsonlite")
install.packages("lubridate")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



###load required packages

In [ ]:
library(httr)
library(dplyr)
library(jsonlite)
library(lubridate)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




###REST API server and Authentication 

In [ ]:
base <- "https://xxx.app:4567/v1.0/"

# the function below get_req sends REST request based on query argument and retrive dataframe

get_req <- function(query) {
  
  s <- paste0(base,query)
  z <- GET(s, content_type("application/json;charset=utf-8"),
           authenticate("user", "pass", "basic") )
  print(s)
  
  print(status_code(z))
  res_text <- content(z, "text", encoding = "UTF-8")
  res_list <-  fromJSON(res_text, flatten = T)
  
}

###User input parameter - start date time and end date time

In [ ]:
StartDate <- "01/03/2020 00:00:01"  # user input : d / m/ y hh:mm:ss
EndDate  <- "17/06/2020 23:59:59" 

###Script to retrive and process data

In [ ]:
  
starttime_numeric <- as.numeric(dmy_hms(StartDate, tz="UTC"))
endtime_numeric <-  as.numeric(dmy_hms(EndDate, tz="UTC")) 

as.POSIXct(starttime_numeric, origin = '1970-01-01', tz="UTC")
as.POSIXct(endtime_numeric, origin = '1970-01-01', tz="UTC")

# get the list of class(fleet type) from OBD

query <- "getClassData/*" 

class_df <- get_req(query)$class

# get the list of vehicles for each of the class(fleet type)

myls <- list()

for(i in 1:nrow(class_df)){
  
  query <- paste0("getVehicleData/",class_df[i,1])
  
  fleet_name <- class_df[i,1]
  
  data_list <- get_req(query)
  
  data_list[[fleet_name]]
  
  df<- cbind( data_list[[fleet_name]], fleet=names(data_list))
  
  myls[[i]] <- df
  
}

vehicle_df <- do.call(rbind, lapply(myls, data.frame))  %>% mutate_if(is.factor, as.character)


# get the defined event definition from OBD by the method - getEventDefinition and extract event by pattern;
# such as "Low Voltage" / "Acceleration" by the user defined function udf_func_EventDef with class(fleet type) agrument

udf_func_EventData <- function(class_name)  {
  
event_ls <- list()  
query <- paste0("getEventDefinition/", class_name)

EventDefinition <-  get_req(query)
EventDefinition[[class_name]]

Event_Name_List <- grep( "Acceleration", EventDefinition[[class_name]]$description, ignore.case = F, value = T )

if(length(Event_Name_List)>0) {

EventDF <-  EventDefinition[[class_name]] %>% filter(description %in% Event_Name_List)

vehicle_by_class <-   vehicle_df  %>% 
                      filter(fleet==class_name) 

event_ls[["EventDefDF"]] <-  EventDF
event_ls[["EventIDArray"]] <- paste(EventDF$eventId, collapse = ",")
event_ls[["VehicleIDArray"]] <- paste(vehicle_by_class$vehicleId, collapse = ",")
event_ls


method_part <- paste0("getEventData/", class_name, "?") 

param_part <- paste0("eventId=",  event_ls[["EventIDArray"]], "&",
                     "vehicleId=", event_ls[["VehicleIDArray"]], "&",
                     "initialDate=", starttime_numeric, "&",
                     "finalDate=", endtime_numeric, "&",
                     "environmentSignals=false")

query <- paste0(method_part, param_part)


raw_obd <-   get_req(query)

if(raw_obd[[1]]$returnCount>0) {
  
  
  print(paste0("Record Found for ", class_name))
   
  data_df <-  raw_obd[[1]]$occurrences

   data_df$startTime <- as.POSIXct(data_df$startTime, origin = '1970-01-01', tz = "UTC") 
   data_df$endTime <- as.POSIXct(data_df$endTime, origin = '1970-01-01', tz = "UTC") 

  data_df$fleet <- class_name

  fina_df <- left_join( data_df,  event_ls[["EventDefDF"]], by=c("eventId"))  %>%
                   left_join(vehicle_df[,c("vehicleId", "name", "fleet")], by=c("vehicleId", "fleet") )
  
} else{ print(paste0("No Record Found for ", class_name))}

}

}

obd_data_list <- list()

for(c in 1:nrow(class_df)){
  
  df_catcher <- udf_func_EventData(class_df$code[c])
  
  #print(class(df_catcher))
  

  if(is.data.frame(df_catcher)==T) {
  
  
  obd_data_list[[c]] <- df_catcher 
   }
  
}


df_final_temp <- do.call(rbind, lapply(obd_data_list, data.frame))  %>% select(-equipmentId)

st_date <-  as.Date(as.POSIXct(starttime_numeric, origin = '1970-01-01')) 
en_date <- as.Date(as.POSIXct(endtime_numeric, origin = '1970-01-01')) 


[1] "2020-03-01 00:00:01 UTC"

[1] "2020-06-17 23:59:59 UTC"

[1] "https://mtm-obd.nomadrail.app:4567/v1.0/getClassData/*"
[1] 200
[1] "https://mtm-obd.nomadrail.app:4567/v1.0/getVehicleData/mtm_xtrapolis"
[1] 200
[1] "https://mtm-obd.nomadrail.app:4567/v1.0/getVehicleData/mtm_comeng"
[1] 200
[1] "https://mtm-obd.nomadrail.app:4567/v1.0/getVehicleData/mtm_siemens"
[1] 200
[1] "https://mtm-obd.nomadrail.app:4567/v1.0/getEventDefinition/mtm_xtrapolis"
[1] 200
[1] "https://mtm-obd.nomadrail.app:4567/v1.0/getEventData/mtm_xtrapolis?eventId=800,899,2810,2811,2888,2889,6481,6482,6483,6484,6485,6486,6487,6488,6489&vehicleId=1301,1302,1303,1304,1305,1306,1307,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319,1320,1321,1322,1323,1324,1325,1326,1327,1328,1329,1330,1331,1332,1333,1334,1335,1336,1337,1338,1339,1340,1341,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384,1385,1386,1387,13

###Write the data to csv file

In [ ]:
write.csv(df_final_temp, paste0("acceleration_", st_date, "_", en_date, ".csv") , row.names = F)